<a href="https://colab.research.google.com/github/Deeksha-Pandit/EVA4-S14/blob/master/Final/DepthMap_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
!git clone https://github.com/Deeksha-Pandit/EVA4-S14.git

Cloning into 'EVA4-S14'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 89 (delta 24), reused 41 (delta 3), pack-reused 0
Unpacking objects: 100% (89/89), done.


In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./EVA4-S14/nyu.h5

--2020-05-07 14:58:50--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.84.122
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.84.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./EVA4-S14/nyu.h5’

./EVA4-S14/nyu.h5   100%[===================>] 164.89M  83.3MB/s    in 2.0s    

2020-05-07 14:58:52 (83.3 MB/s) - ‘./EVA4-S14/nyu.h5’ saved [172897376/172897376]



In [0]:
import zipfile

archive = zipfile.ZipFile('/content/gdrive/My Drive/Mask_Rcnn/Deeksha/data_Part5.zip')

for file in archive.namelist():
    if file.startswith('Overlay5/Fg-Bg/'):
        archive.extract(file, '/content/')

In [0]:
! ls '/content/Overlay5/Fg-Bg/' | wc -l

80000


In [0]:
!ls

EVA4-S14  gdrive  Overlay5  sample_data


In [0]:
%cd /content/EVA4-S14

/content/EVA4-S14


In [0]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import notebook

def DepthNorm(x, maxDepth):
    return maxDepth / x

def predict(model, images, minDepth=10, maxDepth=1000, batch_size=2):
    # Support multiple RGBs, one RGB image, even grayscale 
    if len(images.shape) < 3: images = np.stack((images,images,images), axis=2)
    if len(images.shape) < 4: images = images.reshape((1, images.shape[0], images.shape[1], images.shape[2]))
    # Compute predictions
    predictions = model.predict(images, batch_size=batch_size)
    # Put in expected range
    return np.clip(DepthNorm(predictions, maxDepth=maxDepth), minDepth, maxDepth) / maxDepth

def scale_up(scale, images):
    from skimage.transform import resize
    scaled = []
    
    for i in range(len(images)):
        img = images[i]
        output_shape = (scale * img.shape[0], scale * img.shape[1])
        scaled.append( resize(img, output_shape, order=1, preserve_range=True, mode='reflect', anti_aliasing=True ) )

    return np.stack(scaled)

def load_images(path,start,end):
    loaded_images = []
    for i in range(start,end):
        x = np.clip(np.asarray(Image.open(f"{path}fg-bg{str(i)}.jpg" ).resize((448,448)), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

def to_multichannel(i):
    if i.shape[2] == 3: return i
    i = i[:,:,0]
    return np.stack((i,i,i), axis=2)
        
def display_images(outputs, inputs=None, gt=None, is_colormap=True, is_rescale=True, start = 1):
    import matplotlib.pyplot as plt
    import skimage
    from skimage.transform import resize

    plasma = plt.get_cmap('gray')

    shape = (outputs[0].shape[0], outputs[0].shape[1], 3)
    print(shape)
    
    #all_images = []
    start = start
    for i in notebook.tqdm(range(outputs.shape[0]),desc = f"Loading from {start} images") :
    
        if is_colormap:
            rescaled = outputs[i][:,:,0]
            if is_rescale:
                rescaled = rescaled - np.min(rescaled)
                rescaled = rescaled / np.max(rescaled)
  
            #plt.figure(figsize=(2.24,2.24),dpi=100)
            matplotlib_image = plt.imshow(plasma(rescaled)[:,:,:3])
            pil_image = Image.fromarray(np.uint8( ( matplotlib_image.get_array()*255))).convert("L").resize((224,224))
            pil_image.save(f"/content/Depth_Part5/depth{str(start)}.jpg")
            plt.close() 
            start+=1

            # plt.imsave("a.jpg",plasma(rescaled)[:,:,:3])          

        
           

def save_images(filename, outputs, inputs=None, gt=None, is_colormap=True, is_rescale=False):
    montage =  display_images(outputs, inputs, is_colormap, is_rescale)
    im = Image.fromarray(np.uint8(montage*255))
    im.save(filename)

def load_test_data(test_data_zip_file='nyu_test.zip'):
    print('Loading test data...', end='')
    import numpy as np
    from data import extract_zip
    data = extract_zip(test_data_zip_file)
    from io import BytesIO
    rgb = np.load(BytesIO(data['eigen_test_rgb.npy']))
    depth = np.load(BytesIO(data['eigen_test_depth.npy']))
    crop = np.load(BytesIO(data['eigen_test_crop.npy']))
    print('Test data loaded.\n')
    return {'rgb':rgb, 'depth':depth, 'crop':crop}

def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()
    abs_rel = np.mean(np.abs(gt - pred) / gt)
    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())
    log_10 = (np.abs(np.log10(gt)-np.log10(pred))).mean()
    return a1, a2, a3, abs_rel, rmse, log_10

def evaluate(model, rgb, depth, crop, batch_size=6, verbose=False):
    N = len(rgb)

    bs = batch_size

    predictions = []
    testSetDepths = []
    
    for i in range(N//bs):    
        x = rgb[(i)*bs:(i+1)*bs,:,:,:]
        
        # Compute results
        true_y = depth[(i)*bs:(i+1)*bs,:,:]
        pred_y = scale_up(2, predict(model, x/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0
        
        # Test time augmentation: mirror image estimate
        pred_y_flip = scale_up(2, predict(model, x[...,::-1,:]/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0

        # Crop based on Eigen et al. crop
        true_y = true_y[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        pred_y = pred_y[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        pred_y_flip = pred_y_flip[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        
        # Compute errors per image in batch
        for j in range(len(true_y)):
            predictions.append(   (0.5 * pred_y[j]) + (0.5 * np.fliplr(pred_y_flip[j]))   )
            testSetDepths.append(   true_y[j]   )

    predictions = np.stack(predictions, axis=0)
    testSetDepths = np.stack(testSetDepths, axis=0)

    e = compute_errors(predictions, testSetDepths)

    if verbose:
        print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
        print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))

    return e

In [0]:
import os
import glob
import argparse
import matplotlib

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
#from utils import predict, load_images, display_images
from matplotlib import pyplot as plt


# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model('nyu.h5', custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format("nyu.h5"))

# Input images
div = 200
num = 396601
for  i in range((80000-76600)//div):

  
  inputs = load_images(path ="/content/Overlay5/Fg-Bg/",start=num,end = num+div )
  print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))
# Compute results
  outputs = predict(model, inputs)
# Display results
  display_images(outputs.copy(), inputs.copy(), start = num)
  num+=div

print("done")

Using TensorFlow backend.


Loading model...

Model loaded (nyu.h5).

Loaded (200) images of size (448, 448, 3).
(224, 224, 3)




Loaded (200) images of size (448, 448, 3).
(224, 224, 3)




Loaded (200) images of size (448, 448, 3).
(224, 224, 3)




Loaded (200) images of size (448, 448, 3).
(224, 224, 3)


...



Loaded (200) images of size (448, 448, 3).
(224, 224, 3)




Loaded (200) images of size (448, 448, 3).
(224, 224, 3)




Loaded (200) images of size (448, 448, 3).
(224, 224, 3)




Loaded (200) images of size (448, 448, 3).
(224, 224, 3)



done


In [0]:
!ls Depth_Part5/ | wc -l

ls: cannot access 'Depth_Part5/': No such file or directory
0


In [0]:
%cd /content

/content


In [0]:

from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def main(): 
    # path to folder which needs to be zipped 
    directory = 'Depth_Part5'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('/content/gdrive/My Drive/Mask_Rcnn/Deeksha/Depth_Part5.zip','a') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')         
  
  

main()

Streaming output truncated to the last 5000 lines.
Depth_Part5/depth399565.jpg
Depth_Part5/depth350226.jpg
Depth_Part5/depth320090.jpg
Depth_Part5/depth399784.jpg
Depth_Part5/depth374524.jpg
Depth_Part5/depth344644.jpg
Depth_Part5/depth380089.jpg
Depth_Part5/depth392777.jpg
Depth_Part5/depth349681.jpg
Depth_Part5/depth394355.jpg
Depth_Part5/depth363758.jpg
Depth_Part5/depth385793.jpg
Depth_Part5/depth342793.jpg
Depth_Part5/depth344844.jpg
Depth_Part5/depth378024.jpg
Depth_Part5/depth356008.jpg
Depth_Part5/depth336393.jpg
Depth_Part5/depth349109.jpg
Depth_Part5/depth374050.jpg
Depth_Part5/depth387697.jpg
Depth_Part5/depth352880.jpg
Depth_Part5/depth358055.jpg
Depth_Part5/depth357627.jpg
Depth_Part5/depth365215.jpg
Depth_Part5/depth391393.jpg
Depth_Part5/depth375325.jpg
Depth_Part5/depth376035.jpg
Depth_Part5/depth394631.jpg
Depth_Part5/depth385794.jpg
Depth_Part5/depth358379.jpg
Depth_Part5/depth342216.jpg
Depth_Part5/depth395012.jpg
Depth_Part5/depth365985.jpg
Depth_Part5/depth360677.j